In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%config IPCompleter.greedy = True

In [2]:
df = pd.read_csv("creditcard.csv")
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [3]:
from sklearn.preprocessing import RobustScaler

robust_scaler = RobustScaler()
    
df['Amount_scl'] = robust_scaler.fit_transform(df['Amount'].values.reshape(-1,1))
df['Time_scl'] = robust_scaler.fit_transform(df['Time'].values.reshape(-1,1))
df.drop(columns=['Amount', 'Time'],inplace = True)

In [4]:
df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V22,V23,V24,V25,V26,V27,V28,Class,Amount_scl,Time_scl
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0,1.783274,-0.994983
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,0,-0.269825,-0.994983
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,0,4.983721,-0.994972
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,0,1.418291,-0.994972
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,...,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,0,0.670579,-0.994960


In [5]:
X = df.drop('Class', axis=1)
y = df['Class']

In [6]:
from imblearn.under_sampling import RandomUnderSampler

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
df.Class.value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [9]:
rus = RandomUnderSampler(random_state=42)

In [10]:
X_under, y_under = rus.fit_resample(X,y)

In [11]:
y_under.value_counts()

1    492
0    492
Name: Class, dtype: int64

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X_under, y_under, test_size = 0.2, stratify = y_under,random_state = 42)

In [13]:
y_train.value_counts()

1    394
0    393
Name: Class, dtype: int64

In [14]:

y_test.value_counts()

0    99
1    98
Name: Class, dtype: int64

In [15]:
from sklearn.linear_model import LogisticRegression

In [16]:
lr_clf = LogisticRegression(random_state=42)
lr_clf.fit(X_train, y_train)

LogisticRegression(random_state=42)

In [17]:
y_test_pred = lr_clf.predict(X_test)

In [18]:
from sklearn.metrics import classification_report

In [19]:
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.95      0.95      0.95        99
           1       0.95      0.95      0.95        98

    accuracy                           0.95       197
   macro avg       0.95      0.95      0.95       197
weighted avg       0.95      0.95      0.95       197



In [20]:
from sklearn.metrics import accuracy_score

In [21]:
print(accuracy_score(y_test, y_test_pred))

0.949238578680203


In [22]:
from sklearn.svm import SVC

In [23]:
svc_clf = SVC(kernel='rbf', C=0.1, random_state=42)
svc_clf.fit(X_train,y_train)

SVC(C=0.1, random_state=42)

In [24]:
y_test_pred = svc_clf.predict(X_test)

In [25]:
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.88      0.99      0.93        99
           1       0.99      0.87      0.92        98

    accuracy                           0.93       197
   macro avg       0.94      0.93      0.93       197
weighted avg       0.94      0.93      0.93       197



In [26]:
print(accuracy_score(y_test, y_test_pred))

0.9289340101522843


In [27]:
from sklearn.ensemble import RandomForestClassifier

In [41]:
rdmf_clf = RandomForestClassifier(n_estimators=200, max_depth=3, n_jobs=-1, random_state=42)

In [42]:
rdmf_clf.fit(X_under,y_under)

RandomForestClassifier(max_depth=3, n_estimators=200, n_jobs=-1,
                       random_state=42)

In [43]:
y_test_pred = rdmf_clf.predict(X_test)

In [44]:
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.93      1.00      0.97        99
           1       1.00      0.93      0.96        98

    accuracy                           0.96       197
   macro avg       0.97      0.96      0.96       197
weighted avg       0.97      0.96      0.96       197



In [45]:
print(accuracy_score(y_test, y_test_pred))

0.9644670050761421


In [47]:
from sklearn.model_selection import GridSearchCV

In [61]:
param_grid = {'n_estimators': [2,8,64,100,200,250,500,700,1024],
             'max_depth':[1,2,3,4,5,6,7,8,9,10],
             'n_jobs': [-1],
             'random_state':[42]}

In [62]:
estimator = RandomForestClassifier()
gscv_cls = GridSearchCV(estimator, param_grid, n_jobs=-1)

In [63]:
gscv_cls.fit(X_under, y_under)

GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'n_estimators': [2, 8, 64, 100, 200, 250, 500, 700,
                                          1024],
                         'n_jobs': [-1], 'random_state': [42]})

In [64]:
best_clf = gscv_cls.best_estimator_
best_clf

RandomForestClassifier(max_depth=10, n_estimators=250, n_jobs=-1,
                       random_state=42)

In [65]:
y_test_pred = best_clf.predict(X_test)

In [66]:
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99        99
           1       1.00      0.99      0.99        98

    accuracy                           0.99       197
   macro avg       0.99      0.99      0.99       197
weighted avg       0.99      0.99      0.99       197



In [67]:
print(accuracy_score(y_test, y_test_pred))

0.9949238578680203
